In [1]:
import shodan
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from string import Template
import json
import networkx as nx
import re

If you have a custom dataset, then replace `isp10k.json.gz` below with your dataset's filename.

In [2]:
nodes = set()
edges = set()
isps = {}
ips = {}

for banner in shodan.helpers.iterate_files('isp10k.json.gz'):
    nodes.add(banner['isp'])
    isps[banner['isp']] = isps[banner['isp']] + 1 if isps.get(banner['isp']) else 1
    nodes.add(banner['ip_str'])
    edges.add((banner['ip_str'], banner['isp']))
    
    tags = banner.get('tags') and ' '.join(banner['tags'])
    ips[banner['ip_str']] = (banner['org'], banner['location']['city'], banner['os'], tags)

In [3]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.nx_agraph.graphviz_layout(G)

In [4]:
graph = {'nodes': [], 'edges': []}
for node in nodes:
    color = 'red'
    if node in ips:
        color = 'black'
    
    label = ''
    if node in ips:
        label = node + ' | org: %s, city: %s, os: %s, tags: %s' % ips[node]
    else:
        label = '%s: %d IPs' % (node, isps[node])
    
    graph['nodes'].append({
        'id': node,
        'label': label,
        'x': pos[node][0],
        'y': pos[node][1],
        'color': color
    })

for i, edge in enumerate(edges):
    graph['edges'].append({
        'id': i,
        'source': edge[0],
        'target': edge[1],
        'color': 'rgba(200, 200, 200, .5)'
    })

print(len(graph['nodes']), len(graph['edges']))

13293 13201


In [5]:
js_template = Template('''
graph = $graph_data

S = new sigma({
  graph: graph,
  renderer: {
    container: 'container',
    type: 'canvas'
  },
  settings: {
    labelThreshold: 1e3
  }
})

sigma.plugins.relativeSize(S, 1)

S.refresh()
''')

html_template = Template('''
<!DOCTYPE html>
<html>
  <head>
    <title>IPISP</title>
    <meta charset='utf-8'>
  </head>
  <body>
    <h1 style='font-family:sans-serif;text-align:center'>$title</h1>
    <p style='font-family:sans-serif;text-align:center'>Drag, click, and scroll on the map to interact.</p>
    <br>
    <div id='container' style='height:800px;overflow: hidden'></div>
    <script src='https://cdnjs.cloudflare.com/ajax/libs/sigma.js/1.2.1/sigma.min.js'></script>
    <script src='https://cdnjs.cloudflare.com/ajax/libs/sigma.js/1.2.1/plugins/sigma.plugins.relativeSize.min.js'></script>
    <script>$js_text</script>
  </body>
</html>
''')

In [6]:
js_text = js_template.substitute({
    'graph_data': json.dumps(graph)
})

h = html_template.substitute({
    'title': f'{len(ips):,}' + ' Exposed IPs in the UAE on 4/27/2020',
    'js_text': js_text
})

f = open('index.html', 'w+')
f.write(h)
f.close()

HTML(h)